In [1]:
from hepmc import *
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
def write_events(E_CM, sample, filename):
    n_out = int(sample.data.shape[1]/4)
    
    with open(filename, "w") as f:
        f.write("\nHepMC::Version 2.06.09\nHepMC::IO_GenEvent-START_EVENT_LISTING\n")
        for i in range(sample.size):
            # event
            # E evt_number no_mpi scale alphq_qcd alpha_qed signal_id barcode_signal_process_vertex no_vertices barcode_particle_1 barcode_particle_2 no_random_state {random_state} no_weights {weights}
            f.write("E %i -1 0 1.0000000000000000e+00 1.0000000000000000e+00 0 0 1 10001 10002 0 1 %e\n" % (i, sample.weights[i]))
            
            # weights
            f.write("N 1 \"0\"\n")
            
            # units
            f.write("U GEV MM\n")
            
            # vertex
            # V barcode id x y z ctau no_incoming no_outgoing no_weights {weights}
            f.write("V -1 0 0 0 0 0 2 %i 0\n" % n_out)
            
            # incoming particles
            # P barcode PDG_id px py pz energy gen_mass status_code pol_theta pol_phi barcode_vertex_incoming no_flow {code_index, code}
            f.write("P 10001 11 0 0 %e %e 0 4 0 0 -1 0\n" % (E_CM/2, E_CM/2))
            f.write("P 10002 -11 0 0 %e %e 0 4 0 0 -1 0\n" % (-E_CM/2, E_CM/2))
            
            # outgoing particles
            for j in range(n_out):
                if j == 0:
                    pid = 1
                elif j == 1:
                    pid = -1
                else:
                    pid = 21
                
                E = sample.data[i, 4*j]
                px = sample.data[i, 4*j+1]
                py = sample.data[i, 4*j+2]
                pz = sample.data[i, 4*j+3]
                f.write("P %i %i %e %e %e %e 0 1 0 0 0 0\n" % (10003+j, pid, px, py, pz, E))
            
        f.write("HepMC::IO_GenEvent-END_EVENT_LISTING")

In [3]:
sarge = densities.Sarge(2, 4, 100.)

In [4]:
eeqq2g = densities.ee_qq_ng(100., 2, 5., .3)

SherpaException: [1mSherpa: [0m[34mInitialization_Handler::InitializeTheModel[0m throws [1m[31mmissing module[0m[1m: 
   [0m[31mCannot load model library SherpaSM.[0m

In [ ]:
rambo_mapping = phase_space.RamboOnDiet(100., 4)
mapped = phase_space.MappedDensity(eeqq2g, rambo_mapping)

In [ ]:
%%time
sarge_sample = ImportanceMC(sarge)(eeqq2g.pdf, 10000)
print(sarge_sample.integral, sarge_sample.integral_err)

In [ ]:
#ys = eeqq2g.pdf(sarge_sample.data)
#ratio = ys*sarge_sample.weights
#sarge_sample.weights = ys/sarge_sample.weights
write_events(100., sarge_sample, "../samples/qcd/2-4/sarge_weighted.hepmc")

In [ ]:
np.count_nonzero(sarge_sample.weights)

In [ ]:
bound = sarge_sample.weights.max()

In [ ]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((1000, 16))
for i in range(1000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqq2g.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(1000, 1./1000))

In [ ]:
write_events(100., sarge_sample_unweighted, "../samples/qcd/2-4/sarge_unweighted.hepmc")

In [ ]:
rambo_sample = PlainMC(8)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

In [ ]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [ ]:
%%time
rambo = AcceptRejectSampler(mapped, max_y, 8)
rambo_sample = rambo.sample(1000)

In [ ]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [ ]:
write_events(100., rambo_sample, "../samples/qcd/2-4/rambo.hepmc")

In [ ]:
# just a local sampler (rambo)
met = DefaultMetropolis(8, mapped, proposals.Gaussian(8, .01))
%time met_sample = met.sample(100000, np.random.rand(8))

In [ ]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
write_events(100., met_sample, "../samples/qcd/2-4/metropolis.hepmc")

In [ ]:
# now sarge + local in MC3
importance = DefaultMetropolis(4 * 4, eeqq2g, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(8, mapped, proposals.Gaussian(8, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(8, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
%time mc3_sample = update.sample(100000, np.random.rand(8))

In [ ]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
write_events(100., mc3_sample, "../samples/qcd/2-4/mc3.hepmc")